In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset1=pd.read_csv("shared_articles.csv")
#print(dataset1.shape)
print(dataset1.columns)
dataset1=dataset1[dataset1['eventType']=='CONTENT SHARED']
print(dataset1.shape)
print(dataset1)

Index(['timestamp', 'eventType', 'contentId', 'authorPersonId',
       'authorSessionId', 'authorUserAgent', 'authorRegion', 'authorCountry',
       'contentType', 'url', 'title', 'text', 'lang'],
      dtype='object')
(3047, 13)
       timestamp       eventType            contentId       authorPersonId  \
1     1459193988  CONTENT SHARED -4110354420726924665  4340306774493623681   
2     1459194146  CONTENT SHARED -7292285110016212249  4340306774493623681   
3     1459194474  CONTENT SHARED -6151852268067518688  3891637997717104548   
4     1459194497  CONTENT SHARED  2448026894306402386  4340306774493623681   
5     1459194522  CONTENT SHARED -2826566343807132236  4340306774493623681   
6     1459194557  CONTENT SHARED -2148899391355011268  4340306774493623681   
7     1459194599  CONTENT SHARED  4119190424078847945  4340306774493623681   
8     1459194751  CONTENT SHARED -7926018713416777892  4340306774493623681   
9     1459194842  CONTENT SHARED  3353902017498793780  4340306774493

In [3]:
dataset2=pd.read_csv("users_interactions.csv")
print(dataset2.shape)
print(dataset2.columns)
print(dataset2)

(72312, 8)
Index(['timestamp', 'eventType', 'contentId', 'personId', 'sessionId',
       'userAgent', 'userRegion', 'userCountry'],
      dtype='object')
        timestamp eventType            contentId             personId  \
0      1465413032      VIEW -3499919498720038879 -8845298781299428018   
1      1465412560      VIEW  8890720798209849691 -1032019229384696495   
2      1465416190      VIEW   310515487419366995 -1130272294246983140   
3      1465413895    FOLLOW   310515487419366995   344280948527967603   
4      1465412290      VIEW -7820640624231356730  -445337111692715325   
5      1465413742      VIEW   310515487419366995 -8763398617720485024   
6      1465415950      VIEW -8864073373672512525  3609194402293569455   
7      1465415066      VIEW -1492913151930215984  4254153380739593270   
8      1465413762      VIEW   310515487419366995   344280948527967603   
9      1465413771      VIEW  3064370296170038610  3609194402293569455   
10     1465413864      VIEW   3105154874193

In [4]:
print(dataset1.head(5))

    timestamp       eventType            contentId       authorPersonId  \
1  1459193988  CONTENT SHARED -4110354420726924665  4340306774493623681   
2  1459194146  CONTENT SHARED -7292285110016212249  4340306774493623681   
3  1459194474  CONTENT SHARED -6151852268067518688  3891637997717104548   
4  1459194497  CONTENT SHARED  2448026894306402386  4340306774493623681   
5  1459194522  CONTENT SHARED -2826566343807132236  4340306774493623681   

       authorSessionId authorUserAgent authorRegion authorCountry contentType  \
1  8940341205206233829             NaN          NaN           NaN        HTML   
2  8940341205206233829             NaN          NaN           NaN        HTML   
3 -1457532940883382585             NaN          NaN           NaN        HTML   
4  8940341205206233829             NaN          NaN           NaN        HTML   
5  8940341205206233829             NaN          NaN           NaN        HTML   

                                                 url  \
1  htt

Weighted dict is created to distinguish diff event Type numerically and assigning it to the dataframe.

In [5]:
event_strength_type={
    'VIEW':1.0,
    'LIKE':2.0,
    'BOOKMARK':2.5,
    'FOLLOW':3.0,
    'COMMENT CREATED':4.0
}
dataset2['eventStrength']=dataset2['eventType'].apply(lambda x:event_strength_type[x])
print(dataset2['eventStrength'])

0        1.0
1        1.0
2        1.0
3        3.0
4        1.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       1.0
17       1.0
18       1.0
19       1.0
20       1.0
21       1.0
22       1.0
23       1.0
24       1.0
25       2.5
26       1.0
27       2.5
28       1.0
29       1.0
        ... 
72282    1.0
72283    2.0
72284    1.0
72285    1.0
72286    1.0
72287    2.0
72288    1.0
72289    1.0
72290    1.0
72291    1.0
72292    1.0
72293    1.0
72294    1.0
72295    2.5
72296    1.0
72297    1.0
72298    1.0
72299    1.0
72300    1.0
72301    1.0
72302    1.0
72303    1.0
72304    2.5
72305    1.0
72306    1.0
72307    2.0
72308    1.0
72309    1.0
72310    1.0
72311    1.0
Name: eventStrength, dtype: float64


Here we are eleminating the personId whose interactions is less than 5.That is atleast 5 interactions are there for each unique personId.

In [6]:
count=dataset2.groupby(['personId','contentId']).size().reset_index(name='counts')
#print(count)
count2=count.groupby('personId').size().reset_index(name='counts')#calculating no. of interactions of each person
#print(count2)
count2['mincount']=count2.apply(lambda x:x['counts']>=5,axis=1)#indicating which user has min 5 interactions
#print(count2)
count2=count2[count2['mincount']==True]#eliminating the users having less than 5 interactions
#print(count2)
len(count2)#no. of users having atleast 5 interactions
dataset2['mincount']=dataset2.apply(lambda x:x['personId'] in count2['personId'].values,axis=1)
#print(dataset2)
dataset2_2=dataset2[dataset2['mincount']==True]#finall modified interactions dataframe where users have atleast 5 interactions
print(dataset2_2)
print(len(dataset2_2))

        timestamp eventType            contentId             personId  \
0      1465413032      VIEW -3499919498720038879 -8845298781299428018   
1      1465412560      VIEW  8890720798209849691 -1032019229384696495   
2      1465416190      VIEW   310515487419366995 -1130272294246983140   
3      1465413895    FOLLOW   310515487419366995   344280948527967603   
4      1465412290      VIEW -7820640624231356730  -445337111692715325   
5      1465413742      VIEW   310515487419366995 -8763398617720485024   
6      1465415950      VIEW -8864073373672512525  3609194402293569455   
7      1465415066      VIEW -1492913151930215984  4254153380739593270   
8      1465413762      VIEW   310515487419366995   344280948527967603   
9      1465413771      VIEW  3064370296170038610  3609194402293569455   
10     1465413864      VIEW   310515487419366995  3609194402293569455   
11     1465415229      VIEW  3460026829794173084  1908339160857512799   
12     1465415228      VIEW  3460026829794173084  1

In Deskdrop, users are allowed to view an article many times, and interact with them in different ways (eg. like or comment). Thus, to model the user interest on a given article, we aggregate all the interactions the user has performed in an item by a weighted sum of interaction type strength and apply a log transformation to smooth the distribution.

In [7]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
dataset2_3 = dataset2_2.groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(dataset2_3))
print(dataset2_3.head(10))


# of unique user/item interactions: 39106
              personId            contentId  eventStrength
0 -9223121837663643404 -8949113594875411859       1.000000
1 -9223121837663643404 -8377626164558006982       1.000000
2 -9223121837663643404 -8208801367848627943       1.000000
3 -9223121837663643404 -8187220755213888616       1.000000
4 -9223121837663643404 -7423191370472335463       3.169925
5 -9223121837663643404 -7331393944609614247       1.000000
6 -9223121837663643404 -6872546942144599345       1.000000
7 -9223121837663643404 -6728844082024523434       1.000000
8 -9223121837663643404 -6590819806697898649       1.000000
9 -9223121837663643404 -6558712014192834002       1.584963


splitting the dataset into train and test sets.All evaluation matrix are computed here using the test set.

In [8]:
dataset2_train, dataset2_test = train_test_split(dataset2_3,
                                   stratify=dataset2_3['personId'], 
                                   test_size=0.20,
                                   random_state=42)
print(dataset2_train.shape)
print(dataset2_test.shape)

(31308, 3)
(7798, 3)


In [9]:
dataset2_4=dataset2_3
dataset2_3=dataset2_4.set_index('personId')
dataset2_train=dataset2_train.set_index('personId')
dataset2_test=dataset2_test.set_index('personId')
print(type(dataset2_3))


<class 'pandas.core.frame.DataFrame'>


In [10]:
print(dataset2_train)
print(dataset2_test)

                                contentId  eventStrength
personId                                                
 2002083152443772042  2719909253419802298       1.584963
-2626634673110551643  9032993320407723266       1.000000
 1623838599684589103 -1453783314552286835       1.000000
-714914145425798678  -7722236679806608274       1.000000
 6315091808248158885  2633209697262372913       1.000000
-299809122140089170   3704349024245704692       2.000000
-1578287561410088674  -861913102049789637       2.906891
 1960539917229374622 -5636731245639685654       1.584963
-1443636648652872475 -7349721727282676870       1.000000
-3466039595643284366 -1633984990770981161       1.000000
 2416280733544962613 -5297634649735765167       2.000000
 8879844298911979276 -2808959554282640221       1.000000
 1120069409160402054  5441215535748592870       1.000000
 5660542693104786364  5964888566269323942       1.584963
-89388927330481219   -5148591903395022444       1.584963
-1836083230511905974   95733226

In [11]:
def get_items_interacted(person_id,dataset):
    interacted_items=dataset.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items)==pd.Series else [interacted_items])


In [12]:
x=dataset2_3.index.values
for pid in x[:5]:
    y=get_items_interacted(pid,dataset2_3)
    #print(dataset2.loc[pid]['contentId'])
    print(y)
    #print("\n")

{5211673327552264703, -5002383425685129595, -730957269757756529, 1436883058900979473, -4233177915193302509, -8187220755213888616, -7423191370472335463, -8208801367848627943, -4205346868684833897, -5781461435447152359, 3180828616327439381, 8310689940464940949, -447851796385928420, 7516228655554309785, -2402288292108892893, 3041906492387035176, -6558712014192834002, -8377626164558006982, 1469580151036142903, -6545872007932025533, 834896074125772354, 921770761777842242, 943818026930898372, 6615662748793536194, -3750879736572068916, -6872546942144599345, -6728844082024523434, 8962537427807366481, -7331393944609614247, -4541461982704074404, 5293701842202310496, 7229629480273331039, -6590819806697898649, -3912939678517879962, 4419562057180692966, -6484638837208285334, -559964548932224920, -8949113594875411859, 4563606877148407012, 5087084654882097891, 7734121175534200554, 8302949268716967655, 3367026768872537336}
{5211673327552264703, -5002383425685129595, -730957269757756529, 14368830589009

In [13]:
def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, dataset2_3)
        all_items = set(dataset1['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

In [14]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, dataset2_3)
        all_items = set(dataset1['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
            index=-1
            #index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            for d,c in enumerate(recommended_items):
                if (c==item_id):
                    index=d
                    break
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = dataset2_test.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    dataset2_train),
                                                                                    topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(dataset2_test.index.unique())):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [15]:
item_popularity_df=dataset2_3.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
print(item_popularity_df.head(10))
print(item_popularity_df.shape)

             contentId  eventStrength
0 -4029704725707465084     307.733799
1 -6783772548752091658     233.762157
2  -133139342397538859     228.024567
3 -8208801367848627943     197.107608
4 -6843047699859121724     193.825208
5  8224860111193157980     189.044680
6 -2358756719610361882     183.110951
7  2581138407738454418     180.282876
8  7507067965574797372     179.094002
9  1469580151036142903     170.548969
(2984, 2)


# Popularity Model
The most common approach is the popularity model.It recommends the most popular items to the users.Based on the principle of 'Wisdom of the crowd'.This is a baseline approach model.

In [16]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, dataset1)

In [17]:
x=dataset2_3.index.unique()
for users in x[:5]:
    item_ignore=model_evaluator.get_not_interacted_items_sample(users, 10, seed=42)
    print(item_ignore)
    print(type(item_ignore))
    print("\n\n\n")
    
for users in x[:5]:
    item_ignore=model_evaluator.get_not_interacted_items_sample(users, 10, seed=42)
    item_ignore=list(item_ignore)
    recommend=popularity_model.recommend_items(users,item_ignore)
    print(recommend)
    print("\n\n\n")
    

{-7484960773951920439, 8769121796537771237, -8971085667828564532, 4575212535548313873, -2328163218997425612, 8742078838645536785, -9172673334835262304, 7532891417681708371, -5525441367600219076, 751528815916485437}
<class 'set'>




{-6545872007932025533, -3750879736572068916, 7932334249452694632, -2703878216681968882, -4571929941432664145, 6104598399077154797, -4195851681734441295, 4575212535548313873, 2573252627510191315, -7447329442374055391}
<class 'set'>




{-6545872007932025533, 5183878957903248326, 7932334249452694632, -2703878216681968882, -6035458164887182128, -5283011611531067471, -4195851681734441295, 4575212535548313873, 2543311963908571128, -7447329442374055391}
<class 'set'>




{-6545872007932025533, 3980405883168341377, 7932334249452694632, -2703878216681968882, -4195851681734441295, -4037032837327175755, -7120369133608664808, 7655627888058923987, 2573252627510191315, -7447329442374055391}
<class 'set'>




{-6545872007932025533, 7932334249452694632, -27038782166819688

In [18]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
print(pop_detailed_results_df.head(10))

Evaluating Popularity recommendation model...
1139 users processed

Global metrics:
{'recall@10': 0.36971018209797385, 'recall@5': 0.2436522185175686, 'modelName': 'Popularity'}
              _person_id  hits@10_count  hits@5_count  interacted_count  \
79   3609194402293569455             52            26               192   
10  -2626634673110551643             28            19               134   
26  -1032019229384696495             28            12               130   
7   -1443636648652872475             18             7               117   
66  -2979881261169775358             31            22                87   
224 -3596626804281480007             11             7                80   
150  1116121227607581999             23            13                73   
100   692689608292948411             23            16                69   
28   3636910968448833585             31            21                68   
159 -9016528795238256703             20            14                68 

# Observation
The global metrics shows that recall@10 is 0.36971 means nearly 37% of the items recommended by the Popularity model was among the interacted items of the user in the test set.Similarly 24% of the interacted items was in the top 5 recommended list of the Popularity model.

The diff steps followed in the evaluation of the popularity model are:

1-First a pandas df is created which consists of the list of all the articles in descending order based on their eventStrength score.

2-Now the model evaluator is called on the popularity model.

3-Now items are recommended on the modified interactions_test dataset ignoring the items that are already in the train set.

4-Now its checked whether the item interacted by the user is in the top 5 and top 10 in the recommended list returned by the Popularity model.This way recall@5 and recall@10 are calculated.


In [19]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = dataset1['contentId'].tolist()
print(type(item_ids))
tfidf_matrix = vectorizer.fit_transform(dataset1['title'] + "" + dataset1['text'])
tfidf_feature_names = vectorizer.get_feature_names()
#print(tfidf_matrix)
#print(tfidf_matrix.shape)
#print(tfidf_feature_names)
#print(vectorizer.vocabulary)

<class 'list'>


In [20]:
#returns the sparse tf_idf matrix for a single item_id(content_id) which is considered as the document in this case
def get_item_profile(item_id):
    idx = item_ids.index(item_id)#returns the index of the item_id present in item_ids
    item_profile = tfidf_matrix[idx:idx+1]#returns all the entries(sparse-matrix) of the tf_idf matrix where the document no. is idx
    return item_profile

#returns the sparse_matrix ordered vertically for a bunch of documents(content_id represented as item_id)
def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)#stack the sparse matrices vertically
    return item_profiles

#returns the sparse_matrix for all the content_id that a person_id has interacted with
def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    '''user_item_strengths find the strength of each feature for each person_id.Here first it multiplies
       eventstrength with each of the content_id's features and then sums of the content_id's strength for
       each feature.hence the user_item_strength has (1,5000) shape.calculated strength of each feature for each user'''
    
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    #then the array is normalized to make the calculations easy.
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = dataset2_4[dataset2_4['contentId'] \
                                                   .isin(dataset1['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [34]:
user_profiles = build_users_profiles()
len(user_profiles)
print(user_profiles[-6174073684310263806].shape)


(1, 5000)


In [22]:
idx=user_profiles.keys()
for users in idx:
    x=user_profiles[users]
    y=x[0]
    print(len(y))
    print("\n")

5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000


5000



In [30]:
item_profile=tfidf_matrix[0]
print(item_profile.shape)
print(tfidf_matrix.shape)

(1, 5000)
(3047, 5000)


In [24]:
myprofile = user_profiles[-1479311724257856983]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 5000)


,token,relevance
0,learning,0.305655
1,machine learning,0.255557
2,machine,0.246095
3,google,0.208590
4,data,0.172509
5,ai,0.136818
6,algorithms,0.102396
7,graph,0.098438
8,like,0.096970
9,language,0.083993


In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)#comparing the user's calculated features with
        #each of the document features
        print(cosine_similarities.shape)
        
        #Gets the top similar items in ascending order
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])#arange the 
        #(items_id[i],cosine_similarity[0,i]) in decreasing order of cosine_similarity value.The one with highest cosine
        #similarity is on the top.
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(dataset1)

In [28]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)



Evaluating Content-Based Filtering model...
1139 users processed

Global metrics:
{'recall@10': 0.5320595024365222, 'recall@5': 0.41985124390869455, 'modelName': 'Content-Based'}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
79,3609194402293569455,32,16,192,0.166667,0.083333
10,-2626634673110551643,26,12,134,0.194030,0.089552
26,-1032019229384696495,35,21,130,0.269231,0.161538
7,-1443636648652872475,57,39,117,0.487179,0.333333
66,-2979881261169775358,16,8,87,0.183908,0.091954
224,-3596626804281480007,33,22,80,0.412500,0.275000
150,1116121227607581999,8,5,73,0.109589,0.068493
100,692689608292948411,22,16,69,0.318841,0.231884
28,3636910968448833585,19,7,68,0.279412,0.102941
159,-9016528795238256703,7,2,68,0.102941,0.029412


IN case of Content Based Filtering,the recall@5 is 0.41985 and recall@10 is 0.53205 which is a great improvement thn the performance of popularity model.
